In [1]:
#importing libraries
import pandas as pd
import numpy as np
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
!pip install datasets
from datasets import Dataset, DatasetDict
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from transformers import EvalPrediction
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import numpy as np
from sklearn.metrics import classification_report
from collections import Counter

In [2]:
file_path = "data/EN/subtask-2-annotations.txt"
df = pd.read_csv(file_path, sep="\t", header=None, names=["Document_ID", "High_Level_Narratives", "Sub_Narratives"])

In [3]:
# Group by Document_ID and create lists of narratives
df = df.groupby("Document_ID").agg({
    "High_Level_Narratives": lambda x: list(set(";".join(x).split(";"))),
    "Sub_Narratives": lambda x: list(set(";".join(x).split(";")))
}).reset_index()

df.rename(columns={
    "High_Level_Narratives": "High_Level_Narratives_List",
    "Sub_Narratives": "Sub_Narratives_List"
}, inplace=True)


In [4]:

documents_folder = 'data/EN/raw-documents'


def load_documents(folder_path):
    documents = {}
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):  # Ensure only text files are read
            doc_id = filename  # Extract Document_ID from filename
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                documents[doc_id] = file.read().strip()
    return documents

# Load document texts into a dictionary
document_texts = load_documents(documents_folder)

# Map document texts to the DataFrame
df['Text'] = df['Document_ID'].map(document_texts)

# Check if all documents are successfully mapped
print(f"Number of missing documents: {df['Text'].isnull().sum()}")
df


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of missing documents: 0


,Document_ID,High_Level_Narratives_List,Sub_Narratives_List,Text
0,EN_CC_100000.txt,[CC: Criticism of institutions and authorities...,[CC: Hidden plots by secret schemes of powerfu...,Pentagon plans to serve LAB-GROWN MEAT to troo...
1,EN_CC_100002.txt,[CC: Hidden plots by secret schemes of powerfu...,[CC: Hidden plots by secret schemes of powerfu...,Oxford Residents Mount Resistance Against the ...
2,EN_CC_100003.txt,"[CC: Criticism of climate movement, CC: Critic...",[CC: Criticism of institutions and authorities...,"Fonda Heads To Canada For Oil Sands Protest, M..."
3,EN_CC_100004.txt,"[CC: Criticism of climate movement, CC: Contro...",[CC: Criticism of climate movement: Climate mo...,A Tesla Owner Just Exposed A Sick Secret About...
4,EN_CC_100005.txt,[CC: Criticism of climate movement],"[CC: Criticism of climate movement: Other, CC:...",Climate Crazies Fail in Attempt to Vandalize A...
...,...,...,...,...
394,EN_UA_DEV_100028.txt,[URW: Negative Consequences for the West],[URW: Negative Consequences for the West: Othe...,European gas prices surge 20% as Russia's late...
395,EN_UA_DEV_216.txt,"[URW: Negative Consequences for the West, URW:...","[URW: Discrediting the West, Diplomacy: The EU...","EU 'biggest loser' in Ukraine conflicts, Hunga..."
396,EN_UA_DEV_23.txt,"[URW: Praise of Russia, URW: Amplifying war-re...",[URW: Praise of Russia: Praise of Russian mili...,What is the current trajectory of the evil emp...
397,EN_UA_DEV_24.txt,"[URW: Negative Consequences for the West, URW:...","[URW: Discrediting the West, Diplomacy: Diplom...",Europe ‘Shot Itself in the Lungs’ With Sanctio...


In [5]:
import pandas as pd


# Assign labels based on Document_ID
def assign_label(row):
    if row['Document_ID'].startswith("EN_CC"):
        return 1 if "CC:" in str(row['High_Level_Narratives_List']) else 0  # CC or Other
    elif row['Document_ID'].startswith("EN_UA") or row['Document_ID'].startswith("EN_UA_DEV"):
        return 2 if "URW:" in str(row['High_Level_Narratives_List']) else 0  # URW or Other
    else:
        return 0  # Default to Other

# Add the label column
df['label'] = df.apply(assign_label, axis=1)

print(df)


              Document_ID                         High_Level_Narratives_List  \
0        EN_CC_100000.txt  [CC: Criticism of institutions and authorities...   
1        EN_CC_100002.txt  [CC: Hidden plots by secret schemes of powerfu...   
2        EN_CC_100003.txt  [CC: Criticism of climate movement, CC: Critic...   
3        EN_CC_100004.txt  [CC: Criticism of climate movement, CC: Contro...   
4        EN_CC_100005.txt                [CC: Criticism of climate movement]   
..                    ...                                                ...   
394  EN_UA_DEV_100028.txt          [URW: Negative Consequences for the West]   
395     EN_UA_DEV_216.txt  [URW: Negative Consequences for the West, URW:...   
396      EN_UA_DEV_23.txt  [URW: Praise of Russia, URW: Amplifying war-re...   
397      EN_UA_DEV_24.txt  [URW: Negative Consequences for the West, URW:...   
398      EN_UA_DEV_26.txt                        [URW: Russia is the Victim]   

                                   Sub_

In [6]:
df

,Document_ID,High_Level_Narratives_List,Sub_Narratives_List,Text,label
0,EN_CC_100000.txt,[CC: Criticism of institutions and authorities...,[CC: Hidden plots by secret schemes of powerfu...,Pentagon plans to serve LAB-GROWN MEAT to troo...,1
1,EN_CC_100002.txt,[CC: Hidden plots by secret schemes of powerfu...,[CC: Hidden plots by secret schemes of powerfu...,Oxford Residents Mount Resistance Against the ...,1
2,EN_CC_100003.txt,"[CC: Criticism of climate movement, CC: Critic...",[CC: Criticism of institutions and authorities...,"Fonda Heads To Canada For Oil Sands Protest, M...",1
3,EN_CC_100004.txt,"[CC: Criticism of climate movement, CC: Contro...",[CC: Criticism of climate movement: Climate mo...,A Tesla Owner Just Exposed A Sick Secret About...,1
4,EN_CC_100005.txt,[CC: Criticism of climate movement],"[CC: Criticism of climate movement: Other, CC:...",Climate Crazies Fail in Attempt to Vandalize A...,1
...,...,...,...,...,...
394,EN_UA_DEV_100028.txt,[URW: Negative Consequences for the West],[URW: Negative Consequences for the West: Othe...,European gas prices surge 20% as Russia's late...,2
395,EN_UA_DEV_216.txt,"[URW: Negative Consequences for the West, URW:...","[URW: Discrediting the West, Diplomacy: The EU...","EU 'biggest loser' in Ukraine conflicts, Hunga...",2
396,EN_UA_DEV_23.txt,"[URW: Praise of Russia, URW: Amplifying war-re...",[URW: Praise of Russia: Praise of Russian mili...,What is the current trajectory of the evil emp...,2
397,EN_UA_DEV_24.txt,"[URW: Negative Consequences for the West, URW:...","[URW: Discrediting the West, Diplomacy: Diplom...",Europe ‘Shot Itself in the Lungs’ With Sanctio...,2


In [7]:
# def calculate_class_weights(labels):
#     label_counts = np.bincount(labels)
#     total_samples = len(labels)
#     num_classes = len(label_counts)
#     class_weights = [total_samples / (num_classes * count) if count > 0 else 0 for count in label_counts]
#     return torch.tensor(class_weights, dtype=torch.float)


In [8]:
# class WeightedModel(torch.nn.Module):
#     def __init__(self, base_model, class_weights):
#         super(WeightedModel, self).__init__()
#         self.base_model = base_model
#         self.class_weights = class_weights

#     def forward(self, input_ids, attention_mask, labels=None):
#         outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
#         logits = outputs.logits

#         if labels is not None:
#             loss_fn = torch.nn.CrossEntropyLoss(weight=self.class_weights.to(logits.device))
#             loss = loss_fn(logits, labels)
#             return {"loss": loss, "logits": logits}
#         return {"logits": logits}

In [9]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['Text'], df['label'], test_size=0.2, random_state=42
)


In [10]:
train_texts

,Text
3,A Tesla Owner Just Exposed A Sick Secret About...
18,FACT CHECK: Viral X Image Purporting To Show G...
377,Report: Russia Ramps Up Use of Chem Weapons in...
248,Putin Says West Prepared To Scapegoat Zelensky...
177,"Ukraine Appeals for Calm, Says Russian Invasio..."
...,...
71,Biden Unveils Strict Auto Standards to Speed E...
106,Biden WH ‘Amplifies’ Climate Extremist Group ‘...
270,UK Intelligence Experts Concerned Russia Will ...
348,Trump Predicts the End of US Democracy If He L...


In [11]:
from datasets import Dataset

# Convert train and test data to Hugging Face Datasets
train_dataset = Dataset.from_pandas(pd.DataFrame({
    "text": train_texts,
    "labels": train_labels,
    "Document_ID": df.loc[train_texts.index, "Document_ID"]
}))
test_dataset = Dataset.from_pandas(pd.DataFrame({
    "text": test_texts,
    "labels": test_labels,
    "Document_ID": df.loc[test_texts.index, "Document_ID"]
}))


# Create DatasetDict
datasets = DatasetDict({"train": train_dataset, "test": test_dataset})


In [12]:
train_dataset

Dataset({
    features: ['text', 'labels', 'Document_ID', '__index_level_0__'],
    num_rows: 319
})

In [13]:
test_dataset

Dataset({
    features: ['text', 'labels', 'Document_ID', '__index_level_0__'],
    num_rows: 80
})

In [14]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    # Adjust predictions based on document ID rules
    adjusted_preds = []
    for i, pred in enumerate(predictions):
        doc_id = tokenized_datasets["test"]["Document_ID"][i]  # Access metadata
        if doc_id.startswith("EN_CC") and pred == 2:  # Invalid URW for EN_CC
            adjusted_preds.append(1)  # Switch to CC
        elif (doc_id.startswith("EN_UA") or doc_id.startswith("EN_UA_DEV")) and pred == 1:  # Invalid CC for EN_UA
            adjusted_preds.append(2)  # Switch to URW
        else:
            adjusted_preds.append(pred)

    # Compute metrics
    precision = precision_score(labels, adjusted_preds, average="weighted")
    recall = recall_score(labels, adjusted_preds, average="weighted")
    f1 = f1_score(labels, adjusted_preds, average="weighted")
    return {"precision": precision, "recall": recall, "f1": f1}


In [15]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
base_model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=3)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# labels = df['label'].values
# class_weights = calculate_class_weights(labels)

In [17]:
# weighted_model = WeightedModel(base_model, class_weights)

In [18]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/319 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

In [19]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [20]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=20,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=3,
    seed=42,
    weight_decay=0.01,
    fp16=True
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [21]:
trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

<ipython-input-21-21730b01a56b>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
trainer.train()


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.598241,0.686518,0.687500,0.668947
2,0.822400,0.620548,0.783602,0.775000,0.771429
3,0.504400,0.838042,0.830500,0.825000,0.821181
4,0.238200,1.073788,0.840206,0.800000,0.800455
5,0.142100,0.872721,0.855598,0.850000,0.851242
6,0.142100,0.927599,0.872412,0.862500,0.864005
7,0.018700,1.139127,0.824635,0.825000,0.824083
8,0.018100,1.300028,0.836732,0.825000,0.826335
9,0.021900,1.347218,0.812518,0.812500,0.809027
10,0.009100,1.175949,0.833979,0.825000,0.827112


TrainOutput(global_step=1600, training_loss=0.11105541724013164, metrics={'train_runtime': 553.1296, 'train_samples_per_second': 11.534, 'train_steps_per_second': 2.893, 'total_flos': 1678663605104640.0, 'train_loss': 0.11105541724013164, 'epoch': 20.0})

In [23]:
metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 0.9275986552238464, 'eval_precision': 0.8724116161616161, 'eval_recall': 0.8625, 'eval_f1': 0.864005481381253, 'eval_runtime': 0.7828, 'eval_samples_per_second': 102.203, 'eval_steps_per_second': 25.551, 'epoch': 20.0}
